# NB03: Retrospective Validation — Ecotype Species Screening

**Run on**: Local machine

## Goal

Test **H2**: Does branch distance CV outperform the ANI-based proxy used in `ecotype_analysis` for predicting which species show significant environment–gene content correlation?

The prior project (`ecotype_analysis/data/ecotype_correlation_results.csv`) contains 213 species with Jaccard-based correlation scores (`r_ani_jaccard`, `r_emb_jaccard`). We link these to our new branch-distance CV scores for the 213 species that overlap with the 338-species tree universe.

## Columns from prior analysis

| Column | Description |
|--------|-------------|
| `r_ani_jaccard` | Mantel-like correlation between ANI distance and gene content Jaccard distance |
| `r_emb_jaccard` | Correlation between AlphaEarth embedding distance and gene content Jaccard |
| `r_partial_emb_jaccard` | Partial correlation (env | phylo) — the key ecotype signal |
| `p_partial_emb` | p-value for partial correlation |

## H2 Test

For species in both datasets: does `cv_branch_dist` (new) better predict `r_partial_emb_jaccard` (prior ecotype signal) than `r_ani_jaccard` (old phylo proxy)?

We also test whether composite score predicts prior ecotype signal strength.

In [ ]:
# Cell 1: Imports and load data

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

DATA_PATH = "../data"
FIG_PATH = "../figures"

scored = pd.read_csv(f"{DATA_PATH}/species_scored.csv")
prior = pd.read_csv("../../ecotype_analysis/data/ecotype_correlation_results.csv")

print(f"Scored species: {len(scored)}")
print(f"Prior ecotype_analysis species: {len(prior)}")
print(f"\nPrior columns: {prior.columns.tolist()}")
print(f"\nPrior data types and null counts:")
print(prior.dtypes)
print(prior.isna().sum())

In [ ]:
# Cell 2: Merge scored species with prior results

overlap = scored.merge(
    prior.rename(columns={'species': 'gtdb_species_clade_id'}).drop(columns=['short_name', 'n_genomes'], errors='ignore'),
    on='gtdb_species_clade_id',
    how='inner'
)

print(f"Species in both datasets: {len(overlap)}")
print(f"  With r_partial_emb_jaccard: {overlap['r_partial_emb_jaccard'].notna().sum()}")
print(f"  With r_ani_jaccard: {overlap['r_ani_jaccard'].notna().sum()}")
print(f"  With r_emb_jaccard: {overlap['r_emb_jaccard'].notna().sum()}")

h2_df = overlap.dropna(subset=['cv_branch_dist', 'r_ani_jaccard'])
print(f"
Usable for H2 comparison (cv + r_ani_jaccard): {len(h2_df)}")

h2_partial = overlap.dropna(subset=['cv_branch_dist', 'r_partial_emb_jaccard'])
print(f"Usable for partial correlation comparison: {len(h2_partial)}")


In [ ]:
# Cell 3: H2 — Does branch distance CV predict prior ANI-Jaccard correlation?
#
# r_ani_jaccard measures how strongly phylogenetic distance (ANI proxy)
# correlates with gene content distance. High values mean phylogeny
# explains gene content well — a prerequisite for ecotype signal.

# Correlation: cv_branch_dist vs r_ani_jaccard
r_cv_ani, p_cv_ani = stats.spearmanr(
    h2_df['cv_branch_dist'], h2_df['r_ani_jaccard'], nan_policy='omit'
)

print("H2: Does branch distance CV correlate with prior ANI-Jaccard correlation?")
print(f"  N = {len(h2_df)}")
print(f"  Spearman rho (CV vs r_ani_jaccard) = {r_cv_ani:.3f}  (p = {p_cv_ani:.4f})")

# Compare: does the composite score do better?
if len(h2_partial) > 5:
    r_comp_partial, p_comp_partial = stats.spearmanr(
        h2_partial['composite_score'], h2_partial['r_partial_emb_jaccard'], nan_policy='omit'
    )
    print(f"\n  Spearman rho (composite_score vs r_partial_emb_jaccard) = {r_comp_partial:.3f}  (p = {p_comp_partial:.4f})")
    print(f"  N = {len(h2_partial)}")

if p_cv_ani < 0.05:
    print(f"\n  -> Significant correlation. Branch distance CV predicts prior ecotype signal.")
else:
    print(f"\n  -> No significant correlation. H2 not supported — note sparse partial correlation coverage in prior data.")

In [ ]:
# Cell 4: Visualize H2 — CV vs r_ani_jaccard scatter

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Panel A: CV vs r_ani_jaccard
ax = axes[0]
ax.scatter(h2_df['cv_branch_dist'], h2_df['r_ani_jaccard'], alpha=0.6, s=40, color='#5C6BC0')
m, b = np.polyfit(h2_df['cv_branch_dist'].dropna(), h2_df['r_ani_jaccard'].dropna(), 1)
x_line = np.linspace(h2_df['cv_branch_dist'].min(), h2_df['cv_branch_dist'].max(), 100)
ax.plot(x_line, m * x_line + b, 'k--', linewidth=1.5, alpha=0.7)
ax.set_xlabel('Branch Distance CV (new metric)', fontsize=11)
ax.set_ylabel('r_ani_jaccard (prior ecotype signal)', fontsize=11)
ax.set_title(f'H2: CV vs Prior Ecotype Signal\nrho={r_cv_ani:.2f}, p={p_cv_ani:.3f} (N={len(h2_df)})')

# Panel B: Composite score vs r_ani_jaccard
ax = axes[1]
ax.scatter(h2_df['composite_score'], h2_df['r_ani_jaccard'], alpha=0.6, s=40, color='#E64A19')
if len(h2_df) > 3:
    r_c, p_c = stats.spearmanr(h2_df['composite_score'], h2_df['r_ani_jaccard'], nan_policy='omit')
    m2, b2 = np.polyfit(h2_df['composite_score'], h2_df['r_ani_jaccard'], 1)
    x2 = np.linspace(h2_df['composite_score'].min(), h2_df['composite_score'].max(), 100)
    ax.plot(x2, m2 * x2 + b2, 'k--', linewidth=1.5, alpha=0.7)
    ax.set_title(f'Composite Score vs Prior Ecotype Signal\nrho={r_c:.2f}, p={p_c:.3f}')
ax.set_xlabel('Composite Score (new)', fontsize=11)
ax.set_ylabel('r_ani_jaccard (prior ecotype signal)', fontsize=11)

plt.tight_layout()
plt.savefig(f"{FIG_PATH}/h2_retrospective_validation.png", dpi=150, bbox_inches='tight')
plt.show()
print("Saved h2_retrospective_validation.png")

In [ ]:
# Cell 5: Compare score distributions — are high-scorers also high in prior analysis?

# Divide overlap species into high/low scorers by composite score median
median_score = overlap['composite_score'].median()
high_scorers = overlap[overlap['composite_score'] >= median_score]
low_scorers = overlap[overlap['composite_score'] < median_score]

print(f"Median composite score among overlap species: {median_score:.3f}")
print(f"High scorers (>= median): {len(high_scorers)}, Low scorers: {len(low_scorers)}")

# Compare r_ani_jaccard between high and low scorers
high_ani = high_scorers['r_ani_jaccard'].dropna()
low_ani  = low_scorers['r_ani_jaccard'].dropna()

if len(high_ani) > 3 and len(low_ani) > 3:
    u_stat, p_mw = stats.mannwhitneyu(high_ani, low_ani, alternative='greater')
    print(f"\nMann-Whitney U: high vs low scorers on r_ani_jaccard")
    print(f"  High scorers mean r_ani_jaccard: {high_ani.mean():.3f}")
    print(f"  Low scorers mean r_ani_jaccard:  {low_ani.mean():.3f}")
    print(f"  U={u_stat:.0f}, p={p_mw:.4f} (one-sided: high > low)")

print(f"\nTop 10 overlap species by composite score vs their prior r_ani_jaccard:")
print(overlap.nlargest(10, 'composite_score')[
    ['composite_score', 'r_ani_jaccard', 'r_partial_emb_jaccard']
].to_string(index=False))

In [ ]:
# Cell 6: Final candidate table — top 50 overall, annotated with prior results

top50_annotated = scored.head(50).merge(
    prior.rename(columns={'species': 'gtdb_species_clade_id'})[[
        'gtdb_species_clade_id', 'r_ani_jaccard', 'r_partial_emb_jaccard', 'p_partial_emb'
    ]],
    on='gtdb_species_clade_id',
    how='left'
)

# Flag as prior-validated if r_ani_jaccard > 0.5 (strong ecotype signal in prior work)
top50_annotated['prior_validated'] = top50_annotated['r_ani_jaccard'] > 0.5

output_cols = [
    'rank', 'gtdb_species_clade_id', 'GTDB_species', 'no_genomes',
    'cv_branch_dist', 'env_broad_entropy', 'singleton_fraction', 'composite_score',
    'in_prior_ecotype_analysis', 'r_ani_jaccard', 'prior_validated'
]
top50_annotated[output_cols].to_csv(f"{DATA_PATH}/top50_candidates_annotated.csv", index=False)

print("Top 50 candidates (annotated with prior results):")
print(top50_annotated[[
    'rank', 'short_name', 'no_genomes', 'composite_score',
    'in_prior_ecotype_analysis', 'r_ani_jaccard'
]].to_string(index=False))
print(f"\nSaved top50_candidates_annotated.csv")

In [ ]:
# Cell 7: Summary

print("=== RETROSPECTIVE VALIDATION COMPLETE ===")
print()
print(f"H2 (branch CV predicts prior ecotype signal):")
print(f"  Spearman rho = {r_cv_ani:.3f}, p = {p_cv_ani:.4f}")
if p_cv_ani < 0.05:
    print(f"  -> SUPPORTED: branch distance CV correlates with prior ecotype signal.")
else:
    print(f"  -> NOT SUPPORTED at alpha=0.05. Interpret with caution given sparse coverage.")

print()
print("All output files:")
for f in ['species_scored.csv', 'top50_candidates_annotated.csv']:
    fpath = f"{DATA_PATH}/{f}"
    if os.path.exists(fpath):
        print(f"  {f}: OK")

print()
print("All figures:")
for f in os.listdir(FIG_PATH):
    print(f"  {f}")

print()
print("Ready for synthesis. Run /synthesize to generate REPORT.md.")